# Fertility Contagion Model using AgentPy

An agent-based model that simulates the propagation of a disease through a network.

In [2]:
# Import libraries.

# Data preprocessing.
import pandas as pd

# Model design.
import agentpy as ap
import networkx as nx
import numpy as np
import random

# Visualization.
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

## Data Preprocessing

Get numerical values from Pairfam dataset.

In [3]:
# Loading SPSS .SAV files into a Pandas dataframe.
# https://pandas.pydata.org/docs/reference/api/pandas.read_spss.html
pf_1 = pd.read_spss("../data/raw/anchor/anchor1.sav")

In [21]:
# Standard information about anchor.
anchor_fert = pf_1[
    [
        "id", 
        "pid", 
        "wave", 
        "cohort", 
        "sex_gen", 
        "psex_gen", 
        "age", 
        "page", 
        "infertile", 
        "homosex_new", 
        "relstat", 
        "marstat", 
        "pmarstat", 
        "meetdur", 
        "reldur", 
        "cohabdur", 
        "mardur", 
        "pregnant",
        "nkidsbio", 
        "ykage"
    ]
]

# Filter out respondents without partners.
anchor_fert = anchor_fert.dropna(subset="pid")

# Filter out respondents without relationship status.
anchor_fert = anchor_fert.dropna(subset="relstat")

# Filter out anchors without cohabitation.
anchor_fert = anchor_fert[anchor_fert["relstat"].str.contains("COHAB")]

# Filter out anchors who are homosexual or are infertile.
anchor_fert = anchor_fert[anchor_fert["homosex_new"] == "0 Hetero"]
anchor_fert = anchor_fert[anchor_fert["infertile"] == "0 Fertile"]

In [22]:
anchor_fert


,id,pid,wave,cohort,sex_gen,psex_gen,age,page,infertile,homosex_new,relstat,marstat,pmarstat,meetdur,reldur,cohabdur,mardur,pregnant,nkidsbio,ykage
7,1028000.0,1028101.0,1 2008/09,2 1981-1983,2 Female,1 Male,25.0,26.0,0 Fertile,0 Hetero,4 Married COHAB,2 Married/civil union,2 Married/civil union,74.0,42.0,58.0,41.0,0 Not pregnant,0.0,NaN
13,1300000.0,1300101.0,1 2008/09,3 1971-1973,1 Male,2 Female,35.0,39.0,0 Fertile,0 Hetero,3 Never married COHAB,1 Never married,NaN,128.0,89.0,80.0,NaN,0 Not pregnant,0.0,135.0
19,1455000.0,1455101.0,1 2008/09,3 1971-1973,1 Male,2 Female,36.0,34.0,0 Fertile,0 Hetero,4 Married COHAB,2 Married/civil union,2 Married/civil union,168.0,160.0,151.0,160.0,0 Not pregnant,3.0,84.0
20,1624000.0,1624101.0,1 2008/09,3 1971-1973,2 Female,1 Male,35.0,30.0,0 Fertile,0 Hetero,4 Married COHAB,2 Married/civil union,2 Married/civil union,123.0,122.0,106.0,70.0,0 Not pregnant,1.0,21.0
23,1818000.0,1818101.0,1 2008/09,3 1971-1973,1 Male,2 Female,37.0,29.0,0 Fertile,0 Hetero,4 Married COHAB,2 Married/civil union,2 Married/civil union,146.0,132.0,132.0,133.0,0 Not pregnant,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12390,748982000.0,748982101.0,1 2008/09,2 1981-1983,2 Female,1 Male,26.0,27.0,0 Fertile,0 Hetero,3 Never married COHAB,1 Never married,1 Never married,107.0,51.0,27.0,NaN,0 Not pregnant,1.0,21.0
12393,749211000.0,749211101.0,1 2008/09,3 1971-1973,1 Male,2 Female,37.0,39.0,0 Fertile,0 Hetero,4 Married COHAB,2 Married/civil union,2 Married/civil union,211.0,195.0,129.0,49.0,0 Not pregnant,2.0,5.0
12394,749410000.0,749410101.0,1 2008/09,3 1971-1973,2 Female,1 Male,36.0,42.0,0 Fertile,0 Hetero,8 Divorced/separated COHAB,3 Divorced/dissolved civil union,NaN,227.0,60.0,83.0,NaN,0 Not pregnant,2.0,30.0
12395,749432000.0,749432101.0,1 2008/09,2 1981-1983,1 Male,2 Female,27.0,22.0,0 Fertile,0 Hetero,3 Never married COHAB,1 Never married,1 Never married,70.0,68.0,61.0,NaN,0 Not pregnant,1.0,2.0


In [20]:
pd.unique(anchor_fert["infertile"])

['1 Infertile', '0 Fertile', NaN]
Categories (2, object): ['0 Fertile', '1 Infertile']

## Model Definition

We define a new agent type called `Couple` by creating a subclass of `Agent`.

In [128]:
class Couple(ap.Agent): 

    def setup(self, VoC):
        """ Initialize a new variable at agent creation. """

        self.VoC = VoC

In [143]:
class FertModel(ap.Model):
    """ A simple fertility contagion model. """

    def setup(self):

        print(self.p.VoC)
        print(self.p.agents)

        self.agents = ap.AgentList(
            model = self, 
            objs = self.p.agents, 
            cls = Couple, 
            VoC = ap.AttrIter(self.p.VoC)
        )

    def step(self):

        None

    def update(self):

        None
        self.record("Mean Value", np.mean(np.array(self.agents.VoC)))

    def end(self):

        self.agents.record("VoC")

In [144]:
value_list =  [1,2,3,4,5]
type(value_list)

list

In [145]:
np.mean(np.array(value_list))

3.0

In [146]:
parameters = {
    "agents": 5,
    "steps": 5,
    "seed": 42,
    "VoC": value_list
}

model = FertModel(parameters)
results = model.run()

[1, 2, 3, 4, 5]
5
Completed: 5 steps
Run time: 0:00:00.002989
Simulation finished


In [147]:
results

DataDict {
'info': Dictionary with 9 keys
'parameters': 
    'constants': Dictionary with 4 keys
'variables': 
    'FertModel': DataFrame with 1 variable and 6 rows
    'Couple': DataFrame with 1 variable and 5 rows
'reporters': DataFrame with 1 variable and 1 row
}

In [148]:
results.variables.Couple

,,VoC
obj_id,t,
1,5,1
2,5,2
3,5,3
4,5,4
5,5,5


In [149]:
results.variables.FertModel

,Mean Value
t,
0,3.0
1,3.0
2,3.0
3,3.0
4,3.0
5,3.0
